In [1]:
!pip install -r requirements.txt

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%reload_ext autoreload

In [2]:
#lst = list(await wiki_bench.client.client.models.list())[0][1]
#for l in lst:
#    print(l)

NameError: name 'wiki_bench' is not defined

In [1]:
# ДЛЯ ПОДГОТОВКИ ОКРУЖЕНИЯ - КЕШИРОВАНИЯ ЭМБЕДДИНГОВ, РАЗБИЕНИЯ НА СНИППЕТЫ И Т.Д.
# texts_ready - нужно ли скачивать статьи
# window_size - размер сниппета
# emb_ready - нужно ли получать эмбеддинги
# ann_ready - нужно ли получать заранее сгенерированные аннотации на основе заголовков статей

import sys
import logging
import torch
from sentence_transformers import SentenceTransformer

sys.path.append("src")
from wiki_bench import WikiBench
with open('Access_key.txt', 'r', encoding='utf-8') as file:
    url, key = file.read().split()
logging.getLogger("sentence_transformers.SentenceTransformer").setLevel(logging.ERROR)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder = SentenceTransformer("sergeyzh/BERTA").to(device)

wiki_bench = WikiBench(url, key, model_name='RefalMachine/RuadaptQwen2.5-7B-Lite-Beta', device=device, encoder=encoder, pre_load=False)
await wiki_bench.prepare_env(texts_ready=True, window_size=600, emb_ready=True, ann_ready=True)

resource module not available on Windows


Split strings:   0%|          | 0/36860 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/36860 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/36860 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/36860 [00:00<?, ?it/s]

Finding newlines for mmindex:   0%|          | 0.00/87.2M [00:00<?, ?B/s]

In [4]:
import sys

sys.path.append("src")
from wiki_bench import WikiBench
with open('Access_key.txt', 'r', encoding='utf-8') as file:
    url, key = file.read().split()

resource module not available on Windows


In [5]:
import logging
import torch
from sentence_transformers import SentenceTransformer

logging.getLogger("sentence_transformers.SentenceTransformer").setLevel(logging.ERROR)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder = SentenceTransformer("sergeyzh/BERTA").to(device)

In [6]:
bench = WikiBench(
    url, key,
    model_name='yagpt5lite',
    device=device,
    encoder=encoder,
    pre_load=True
)

In [5]:
import asyncio
import json
from datetime import datetime, UTC
import aiofiles
import os


async def run_bench_for_model(url: str,
                              api_key: str,
                              model_name: str,
                              device = None,
                              encoder = None,
                              out_dir: str = "resultsruadapt",
                              reference_mode: int = 1,
                              neighbor_count = 0,
                              description_mode = 1,
                              mode = 1
                             ):

    file_path = f"{out_dir}/{model_name.replace('/', '_')}.jsonl"
    failure_log_path = f"{out_dir}/{model_name.replace('/', '_')}.failures.log"

    os.makedirs(out_dir, exist_ok=True)

    bench = WikiBench(url, api_key, model_name=model_name, device=device, encoder=encoder, pre_load=True)
    
    async with aiofiles.open(file_path, "a", encoding="utf-8") as f, \
               aiofiles.open(failure_log_path, "a", encoding="utf-8") as lf:
        
        async def log_failure(stage, article_name, exc):
            ts = datetime.now(UTC).isoformat(timespec="seconds")
            msg = f"{ts} | {model_name} | {stage} | {article_name} | {type(exc).__name__}: {str(exc)}\n"
            await lf.write(msg)
            await lf.flush()

        # ---------- запросы ----------
        async for article_name, item, annotation in bench.query_stream(reference_mode=reference_mode):
            try:
                await f.write(_as_jsonl(model_name, article_name, "query", item, annotation, False))
            except (KeyboardInterrupt, asyncio.CancelledError):
                raise
            except Exception as e:
                await log_failure("query", article_name, e)
                await asyncio.sleep(120)
                continue
        print(f"[{model_name}] — запросы готовы")
        
        # ---------- планы статей ----------
        async for article_name, item in bench.outline_stream(
            neighbor_count=neighbor_count, 
            description_mode=description_mode, 
            mode=mode
        ):
            try:
                await f.write(_as_jsonl(model_name, article_name, "outline", item, '', False))
            except (KeyboardInterrupt, asyncio.CancelledError):
                raise
            except Exception as e:
                await log_failure("outline", article_name, e)
                await asyncio.sleep(120)
                continue
        print(f"[{model_name}] — планы готовы")
        
        # ---------- секции статей ----------
        async for article_name, item in bench.sections_stream():
            try:
                await f.write(_as_jsonl(model_name, article_name, "section", item, '', True))
            except (KeyboardInterrupt, asyncio.CancelledError):
                raise
            except Exception as e:
                await log_failure("section", article_name, e)
                await asyncio.sleep(120)
                continue
        print(f"[{model_name}] — секции готовы")

    print(f"[{model_name}] — готово. Результаты: {file_path}")
    
def serialize(d):
    return [[[k, n], v] for (k, n), v in d.items()]

def _as_jsonl(model, article_name, phase, data, annotation='', needs_ser=False):
    if needs_ser:
        data = serialize(data)
    entry = {
        "ts": datetime.now(UTC).isoformat(timespec="seconds"),
        "model": model,
        "article_name": article_name,
        "phase": phase,
        "annotation": annotation,
        "data": data,
    }
    return json.dumps(entry, ensure_ascii=False) + "\n"


In [6]:
from openai_utils import AsyncList

async def main():
    models = AsyncList()
    model_names = [
        #'RefalMachine/RuadaptQwen3-32B-Instruct-v2',
        #'Qwen3-235B-A22B',
        #'DeepSeek V3',
        #'tpro',
        'RefalMachine/RuadaptQwen2.5-7B-Lite-Beta'
        #'yagpt5lite'
    ]
    for name in model_names:
        models.append(run_bench_for_model(url, key, name, device, encoder))

    await models.complete_couroutines(batch_size=len(model_names))
    models = await models.to_list()

In [ ]:
await main()

In [ ]:
import sys
import logging
import glob
import os
import json
import torch
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

sys.path.append("src")
from wiki_bench import WikiBench

with open('Access_key.txt', 'r', encoding='utf-8') as f:
    url, key = f.read().split()

logging.getLogger("sentence_transformers.SentenceTransformer").setLevel(logging.ERROR)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = SentenceTransformer("sergeyzh/BERTA").to(device)
bench = WikiBench(
    url, key,
    model_name='yagpt5lite',
    device=device,
    encoder=encoder,
    pre_load=True
)

summary = []

pattern = 'results/*.jsonl'

need_sections_results = True

for filepath in glob.glob(pattern):
    name = os.path.splitext(os.path.basename(filepath))[0]
    print(f"\n=== Обработка файла: {name} ===")

    data = []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))

    data_query   = data[:100]
    data_outline = data[100:200]
    if need_sections_results:
        data_sections= data[200:]

    if need_sections_results:
        data_sec_ref = []
        for item in data_sections:
            d = {(sec[0][0], sec[0][1]): sec[1] for sec in item['data']}
            data_sec_ref.append({'article_name': item['article_name'], 'data': d})
    
        results_sections = []
        for item in tqdm(data_sec_ref, desc="Sections eval", leave=False):
            page = bench.wiki_agent.get_article(
                item['article_name'],
                retrieve_sources=False,
                is_downloaded=True,
                verbose=False,
                html=True,
                needs_saving=False
            )
            results_sections.append(
                await bench.wiki_evaluater.rank_sections(item['data'], page)
            )
        boot_sections = bench.wiki_evaluater.bootstrap(results_sections, is_flat=False)
        print(f"Sections bootstrap: {boot_sections}")

    if need_sections_results:
        summary.append({
            'file': name,
            'query_ndcg': avg_ndcg,
            'query_pr':   avg_pr,
            'outline_bootstrap': boot_outline,
            'sections_bootstrap': boot_sections
        })
    else:
        summary.append({
            'file': name,
            'query_ndcg': avg_ndcg,
            'query_pr':   avg_pr,
            'outline_bootstrap': boot_outline
        })

print()
print("Сводка по всем файлам")
for rec in summary:
    print(f"{rec['file']:30s} | "
          f"NDCG={rec['query_ndcg']:.4f}, PR={rec['query_pr']:.4f} | "
          f"Outline={rec['outline_bootstrap']}", end='')
    if need_sections_results:
        print(f" | Sections={rec['sections_bootstrap']}")
    else:
        print()

In [ ]:
import math
from rouge import Rouge
import sys
import logging
import glob
import os
import json
import torch
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import numpy as np
from scipy.stats import bootstrap
from razdel import tokenize
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

sys.path.append("src")
from wiki_bench import WikiBench

with open('Access_key.txt', 'r', encoding='utf-8') as f:
    url, key = f.read().split()

logging.getLogger("sentence_transformers.SentenceTransformer").setLevel(logging.ERROR)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = SentenceTransformer("sergeyzh/BERTA").to(device)
bench = WikiBench(
    url, key,
    model_name='yagpt5lite',
    device=device,
    encoder=encoder,
    pre_load=True
)

def rouge_L(ref, pred):
    rouge = Rouge()

    score = rouge.get_scores(pred, ref)[0]["rouge-l"]["f"]
    return score


pattern = 'resultsruadapt/*.jsonl'

need_sections_results = False

def calc(data):
        data = np.array(data)
        data1 = (data,)
        bootstrap_ci = bootstrap(data1, np.mean, confidence_level=0.95, n_resamples=len(data)-1)
        
        dist = bootstrap_ci.bootstrap_distribution
        mean = np.quantile(dist, q=0.5)
        min = np.quantile(dist, q=0.025)
        max = np.quantile(dist, q=0.975)
        return mean, min, max

def bootstrap2(data):
    f_mean, f_min, f_max = calc([e for el in data for e in el])
    return f_mean, f_min, f_max

def bleu_score_razdel(ref_text, pred_text):
    ref_tokens = [t.text for t in tokenize(ref_text)]
    pred_tokens = [t.text for t in tokenize(pred_text)]
    smooth = SmoothingFunction().method1

    score = sentence_bleu([ref_tokens], pred_tokens, smoothing_function=smooth)
    return score

for filepath in glob.glob(pattern):
    name = os.path.splitext(os.path.basename(filepath))[0]
    if name != 'RefalMachine_RuadaptQwen2.5-7B-Lite-Beta':
        continue
    print(f"\n=== Обработка файла: {name} ===")
    data = []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))

    data_sec = data[200:]
    r_l_scores_all = []
    data_sec_ref = []
    
    for item in tqdm(data_sec):
        result = {(sec[0][0], sec[0][1]): sec[1] for sec in item['data']}
        r_l_scores = []
        page = bench.wiki_agent.get_article(
            item['article_name'],
            retrieve_sources=False,
            is_downloaded=True,
            verbose=False,
            html=True,
            needs_saving=False
        )
        for section in page.filtered_outline.keys():
            gen_text = result.get(section)
            if not gen_text or gen_text == -1:
                continue
            gold_text = page.filtered_outline[section]
            
            #r_l = rouge_L(gold_text, gen_text)
            r_l = rouge_L(gold_text, gen_text)
            r_l_scores.append(r_l)
        r_l_scores_all.append(r_l_scores)

    print(bootstrap2(r_l_scores_all))
    print()